Downloaing malaria data from kaggle website to a machine provided by colab website.

In [6]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

Saving kaggle.json to kaggle (1).json
cell-images-for-detecting-malaria.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzip the downloaded data by zipfile library, find name of images by os library,read image files by PIL library and finally save them as matrix by numpy. 

In [3]:
from zipfile import ZipFile
import os
from PIL import Image
import numpy as np

zip_ref = ZipFile('/content/cell-images-for-detecting-malaria.zip', 'r')
zip_ref.extractall('/content/Malaria')
zip_ref.close()


img_rows=44
img_cols=44

path1='/content/Malaria/cell_images/Parasitized'
path2='/content/Malaria/cell_images/Uninfected'

listing = os.listdir(path1)
num_samples = len(listing)
print(num_samples)
images=[]
for file in listing:
  
    try:
      im=Image.open(path1 + '/' + file)
      images.append(np.asarray(im.resize((img_rows, img_cols))))
    except:
      continue
images1 = np.array(images)

listing = os.listdir(path2)
num_samples = len(listing)-1
print(num_samples)
images=[]
for file in listing:
    try:
      im=Image.open(path2 + '/' + file)
      images.append(np.asarray(im.resize((img_rows, img_cols))))
    except:
      continue
images2 = np.array(images)

13780
13779


load necessary library for next step. 

In [4]:

from time import time 


# KERAS
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.losses import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.utils import np_utils
from keras import regularizers
from tensorflow.python.keras.callbacks import TensorBoard



import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
# SKLEARN
from sklearn.utils import shuffle

Using TensorFlow backend.


constract lable matrix for softmax layer then merge innfected and unifected image in one matrix. after that normalize, shufle and divide data in three group, train, validation and test. finally train proposed model using data.

In [7]:
label=np.zeros((num_samples*2,2))
label[:num_samples,0]=1
label[num_samples:,1]=1

data=np.zeros((num_samples*2,44,44,3))
data[:num_samples,:,:,:]=images1
data[num_samples:num_samples*2,:,:,:]=images2

data=data/255.0

quee=np.random.permutation(num_samples*2)
label=label[quee,:]
data=data[quee,:,:,:]

print(np.shape(label),np.shape(data))


length=len(label)
train_count=int(length*0.9)
valid_count=int(length*0.05)
test_count=length-train_count-valid_count
train_data=data[:train_count,:,:,:]
train_label=label[:train_count,:]
valid_data=data[train_count:train_count+valid_count,:,:,:]
valid_label=label[train_count:train_count+valid_count,:]
test_data=data[train_count+valid_count:,:,:,:]
test_label=label[train_count+valid_count:,:]

num_classes=2
batch_size=128
epochs=70
l2_penalty_rate = 0.01
FC_neuron_num=128


model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=(44,44,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))


model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(Conv2D(128, (3, 3), activation='linear'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))


model.add(Conv2D(512, kernel_size=(2, 2), strides=(1, 1),
                 activation='relu'))

model.add(Flatten())

model.add(Dense(FC_neuron_num, activation='relu',kernel_regularizer=regularizers.l2(l2_penalty_rate)))
model.add(Dense(FC_neuron_num, activation='relu',kernel_regularizer=regularizers.l2(l2_penalty_rate)))
model.add(Dense(FC_neuron_num, activation='relu',kernel_regularizer=regularizers.l2(l2_penalty_rate)))
model.add(Dense(FC_neuron_num, activation='relu',kernel_regularizer=regularizers.l2(l2_penalty_rate)))
model.add(Dense(num_classes, activation=tf.nn.softmax))
model.summary()

from sklearn.metrics import classification_report, confusion_matrix

adam=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
model.compile(loss=categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy'])


model.fit(train_data, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(valid_data, valid_label))


score = model.evaluate(test_data, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(27558, 2) (27558, 44, 44, 3)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 42, 42, 64)        1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 40, 40, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 18, 18, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_12 (Conv2D)      

Extract feature from last layer of the proposed model in order to use it this new data to train SVM model.

In [0]:
from keras.models import Model
layer_name = 'dense_9'
inter_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
X_train = inter_layer_model.predict(train_data)
X_test = inter_layer_model.predict(test_data)
X_valid = inter_layer_model.predict(valid_data)
y_train = train_label[:,0]
y_test = test_label[:,0]
y_valid = valid_label[:,0]

Train SVM models and find best model using grid search.

In [11]:
from sklearn.svm import SVC
from sklearn import decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

models = Pipeline([('svc', SVC())])

polynomial = {'svc__kernel': ['polynomial'], 'svc__C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10,100,1000],
              'svc__degree': [2,3,4,5], 'svc__coef0': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10,100,1000]}
linear = {'svc__kernel': ['linear'], 'svc__C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10,100,1000]}
rbf = {'svc__kernel': ['rbf'], 'svc__C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10,100,1000],
       'svc__gamma': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10,100, 1000]}
params = []
#params.append(polynomial)
params.append(linear)
#params.append(rbf)
print('h')
grid = GridSearchCV(models, param_grid=params, cv=5)
grid.fit(X_train, y_train)
print('h')
print ("score = %3.2f" %(grid.score(X_test,y_test)))
print (grid.best_params_)


y_pred = grid.predict(X_test)

def normalize(confusion):
    confusion = confusion.astype(float)
    count = np.sum(confusion, axis=1)
    for i in range(len(count)):
        confusion[i,:] = confusion[i,:]/count[i]
    return confusion

print(normalize(confusion_matrix(y_test,y_pred)))
print(classification_report(y_test,y_pred))

h
h
score = 0.96
{'svc__C': 1, 'svc__kernel': 'linear'}
[[0.96226415 0.03773585]
 [0.04202899 0.95797101]]
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96       689
         1.0       0.96      0.96      0.96       690

    accuracy                           0.96      1379
   macro avg       0.96      0.96      0.96      1379
weighted avg       0.96      0.96      0.96      1379

